In [1]:
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister, transpile
from qiskit.circuit.library import QFTGate  
from qiskit_aer import AerSimulator
from qiskit_aer.primitives import Sampler as AerSampler
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager
from qiskit.visualization import plot_histogram
import matplotlib.pyplot as plt
import numpy as np
from qiskit.circuit.library.basis_change import QFT
from neqr import neqr
from ladder import ladder_up
from enhanced_grover import filter_oracle
from qsubtract import subtract
from qadd import qft_adder
from PIL import Image
from qedgenod import qedgevh
import os

In [2]:
num_bits = 5
intensity_bits =  4
dim = 2 ** num_bits

In [3]:
img_file = 'batman16.jpg'
base_name = os.path.splitext(os.path.basename(img_file))[0]
output_dir = f"{base_name}_edges"
os.makedirs(output_dir, exist_ok=True)

In [4]:
def load_image(filepath, size):
    img = Image.open(filepath).convert('L')  # Grayscale
    img = img.resize(size, Image.Resampling.LANCZOS)
    img_array = np.array(img)
    return img_array

def normalize_image(image, bits=intensity_bits):
    max_val = 2**bits - 1
    return np.round((image / 255) * max_val).astype(int)

image = load_image(img_file,size=(dim,dim))
image = normalize_image(image, bits=intensity_bits)
print(image)

[[15 15 15 ... 15 15 15]
 [15 15 15 ... 15 15 15]
 [15 15 15 ... 15 15 15]
 ...
 [15 15 15 ... 15 15 15]
 [15 15 15 ... 15 15 15]
 [15 15 15 ... 15 15 15]]


In [5]:
max_threshold = 2**(intensity_bits+1) - 1

In [ ]:
for t in range(max_threshold):
    threshold = format(t, f'0{intensity_bits + 1}b')
    print(f"\nRunning threshold: {threshold}")
        
        # 3. Run quantum edge detection
    counts = dict(qedgevh(image, threshold, num_bits, intensity_bits))
        
        # 4. Initialize edge image
    dim = 2**num_bits
    edge_img = np.zeros((dim, dim), dtype=int)

        # 5. Parse measurement results
    for qstate, count in counts.items():
        qstate = qstate.replace(" ", "")

        if len(qstate) < 1 + 2 * num_bits:
            continue

        intensity_bin = qstate[:1]
        y_bin = qstate[1:1 + num_bits]
        x_bin = qstate[1 + num_bits:1 + 2*num_bits]

        intensity = int(intensity_bin, 2)
        y = int(y_bin, 2)
        x = int(x_bin, 2)

        edge_img[y, x] = intensity

        # 6. Display output
        plt.imshow(edge_img, cmap='gray_r', interpolation='nearest')
        plt.colorbar(label='Intensity')
        plt.title(f'Edge Filtered Image\nThreshold: {threshold}')
        plt.axis('off')
        out_path = os.path.join(output_dir, f"{base_name}_threshold_{threshold}.png")
        plt.savefig(out_path, bbox_inches='tight')
        plt.close()  # Close the figure to avoid memory bloat 


Running threshold: 00000
5
[0, 0, 0, 0, 0]
4 0
max 4
3 0
2 0
1 0
0 0

Running threshold: 00001
5
[1, 0, 0, 0, 0]
4 0
max 4
3 0
2 0
1 0
0 1

Running threshold: 00010
5
[0, 1, 0, 0, 0]
4 0
max 4
3 0
2 0
1 1
0 0

Running threshold: 00011
5
[1, 1, 0, 0, 0]
4 0
max 4
3 0
2 0
1 1
0 1

Running threshold: 00100
5
[0, 0, 1, 0, 0]
4 0
max 4
3 0
2 1
1 0
0 0

Running threshold: 00101
5
[1, 0, 1, 0, 0]
4 0
max 4
3 0
2 1
1 0
0 1

Running threshold: 00110
5
[0, 1, 1, 0, 0]
4 0
max 4
3 0
2 1
1 1
0 0

Running threshold: 00111
5
[1, 1, 1, 0, 0]
4 0
max 4
3 0
2 1
1 1
0 1

Running threshold: 01000
5
[0, 0, 0, 1, 0]
4 0
max 4
3 1
2 0
1 0
0 0

Running threshold: 01001
5
[1, 0, 0, 1, 0]
4 0
max 4
3 1
2 0
1 0
0 1
